In [2]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import os
import random

tfk = tf.keras
tfkl = tf.keras.layers

In [3]:
seed = 42
batch_size = 32
input_shape = (96, 96, 3)
epochs = 30

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [4]:
dataset_dir = 'dataset96/train'


labels = ['Species1',       # 0
          'Species2',       # 1
          'Species3',       # 2
          'Species4',       # 3
          'Species5',       # 4
          'Species6',       # 5
          'Species7',       # 6
          'Species8',       # 7
          ]


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Create an instance of ImageDataGenerator with Data Augmentation
aug_train_data_gen = ImageDataGenerator(rotation_range=10,
                                        height_shift_range=10,
                                        width_shift_range=10,
                                        zoom_range=0.1,
                                        shear_range = 0.2,
                                        brightness_range=[0.4, 1.5],
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        fill_mode='reflect')

train_val_data_gen = ImageDataGenerator()

# Obtain a data generator with the 'ImageDataGenerator.flow_from_directory' method to create iterator over the indices
aug_train_gen = aug_train_data_gen.flow_from_directory(directory=dataset_dir,
                                                        target_size=(96, 96),
                                                        color_mode='rgb',
                                                        classes=None,
                                                        class_mode='categorical',
                                                        batch_size=batch_size,
                                                        shuffle=True,#at the end of the epoch it will shuffle the data
                                                        seed=seed,
                                                        )

val_gen = train_val_data_gen.flow_from_directory(directory="dataset96/val",
                                                  target_size=(96, 96),
                                                  color_mode='rgb',
                                                  classes=None,
                                                  class_mode='categorical',
                                                  batch_size=batch_size,
                                                  shuffle=True, 
                                                  seed=seed,
                                                  )

Found 2829 images belonging to 8 classes.
Found 713 images belonging to 8 classes.


In [10]:
# Download and plot the Xception model
base_model = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    classes=8, # number of classes to classify images into
    classifier_activation="softmax",
)

In [11]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(8, activation='softmax')(x)

# Connect input and output through the Model class
model = tfk.Model(inputs=base_model.input, outputs=predictions, name='model')


In [12]:
class_weights = {0: 2.389358108108108, 
                 1: 0.8320588235294117, 
                 2: 0.8583131067961165, 
                 3: 0.8667279411764706, 
                 4: 0.8340212264150944, 
                 5: 1.9978813559322033, 
                 6: 0.8243006993006993, 
                 7: 0.8709975369458128}

In [14]:
callbacks = []

# Tensorboard
tensorboard = TensorBoard(log_dir="tb_logs/")
callbacks.append(tensorboard)
# Early Stopping
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,
                                                restore_best_weights=True)
callbacks.append(es_callback)

# Compile the model
model.compile(loss=tfk.losses.CategoricalCrossentropy(), optimizer=tfk.optimizers.Adam(), metrics='accuracy')
model.summary()

# Train the model
model.fit(
    x=aug_train_gen,
    batch_size=batch_size,
    epochs=epochs,
    class_weight=class_weights,
    validation_data=val_gen,
    callbacks= callbacks)


model.save("data_augmentation_experiments/CNN_xception_Best")

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 96, 96, 3)]  0           []                               
                                                                                                  
 block1_conv1 (Conv2D)          (None, 47, 47, 32)   864         ['input_2[0][0]']                
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 47, 47, 32)  128         ['block1_conv1[0][0]']           
 ation)                                                                                           
                                                                                                  
 block1_conv1_act (Activation)  (None, 47, 47, 32)   0           ['block1_conv1_bn[0][0]']    

                                                                                                  
 block4_sepconv2 (SeparableConv  (None, 12, 12, 728)  536536     ['block4_sepconv2_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block4_sepconv2_bn (BatchNorma  (None, 12, 12, 728)  2912       ['block4_sepconv2[0][0]']        
 lization)                                                                                        
                                                                                                  
 conv2d_6 (Conv2D)              (None, 6, 6, 728)    186368      ['add_13[0][0]']                 
                                                                                                  
 block4_pool (MaxPooling2D)     (None, 6, 6, 728)    0           ['block4_sepconv2_bn[0][0]']     
          

 block7_sepconv1_bn (BatchNorma  (None, 6, 6, 728)   2912        ['block7_sepconv1[0][0]']        
 lization)                                                                                        
                                                                                                  
 block7_sepconv2_act (Activatio  (None, 6, 6, 728)   0           ['block7_sepconv1_bn[0][0]']     
 n)                                                                                               
                                                                                                  
 block7_sepconv2 (SeparableConv  (None, 6, 6, 728)   536536      ['block7_sepconv2_act[0][0]']    
 2D)                                                                                              
                                                                                                  
 block7_sepconv2_bn (BatchNorma  (None, 6, 6, 728)   2912        ['block7_sepconv2[0][0]']        
 lization)

                                                                                                  
 block10_sepconv1_act (Activati  (None, 6, 6, 728)   0           ['add_19[0][0]']                 
 on)                                                                                              
                                                                                                  
 block10_sepconv1 (SeparableCon  (None, 6, 6, 728)   536536      ['block10_sepconv1_act[0][0]']   
 v2D)                                                                                             
                                                                                                  
 block10_sepconv1_bn (BatchNorm  (None, 6, 6, 728)   2912        ['block10_sepconv1[0][0]']       
 alization)                                                                                       
                                                                                                  
 block10_s

 v2D)                                                                                             
                                                                                                  
 block12_sepconv3_bn (BatchNorm  (None, 6, 6, 728)   2912        ['block12_sepconv3[0][0]']       
 alization)                                                                                       
                                                                                                  
 add_22 (Add)                   (None, 6, 6, 728)    0           ['block12_sepconv3_bn[0][0]',    
                                                                  'add_21[0][0]']                 
                                                                                                  
 block13_sepconv1_act (Activati  (None, 6, 6, 728)   0           ['add_22[0][0]']                 
 on)                                                                                              
          

Epoch 15/30
89/89 [==============================] - 144s 2s/step - loss: 0.3360 - accuracy: 0.8908 - val_loss: 0.9196 - val_accuracy: 0.7335
Epoch 16/30
89/89 [==============================] - 160s 2s/step - loss: 0.3724 - accuracy: 0.8738 - val_loss: 1.0944 - val_accuracy: 0.6508
Epoch 17/30
89/89 [==============================] - 139s 2s/step - loss: 0.3062 - accuracy: 0.9074 - val_loss: 1.5858 - val_accuracy: 0.6746
Epoch 18/30
89/89 [==============================] - 143s 2s/step - loss: 0.3233 - accuracy: 0.8904 - val_loss: 1.6338 - val_accuracy: 0.6592
Epoch 19/30
89/89 [==============================] - 145s 2s/step - loss: 0.2989 - accuracy: 0.9081 - val_loss: 1.2659 - val_accuracy: 0.6760
Epoch 20/30
89/89 [==============================] - 144s 2s/step - loss: 0.2791 - accuracy: 0.9092 - val_loss: 1.4878 - val_accuracy: 0.6676
Epoch 21/30
89/89 [==============================] - 130s 1s/step - loss: 0.2482 - accuracy: 0.9130 - val_loss: 2.6852 - val_accuracy: 0.5582


INFO:tensorflow:Assets written to: data_augmentation_experiments/CNN_xception_Best\assets


INFO:tensorflow:Assets written to: data_augmentation_experiments/CNN_xception_Best\assets
